In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import requests

# Wikipedia日本語版のAPIエンドポイント
WIKI_API_URL = "https://ja.wikipedia.org/w/api.php"

# 環境変数の取得
load_dotenv("../.env")

# モデル名
MODEL_NAME = "gpt-4o-mini"

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# テストコードで確認
keyword = "NBA"

# 検索キーワードに該当するWikipediaページを取得する関数
def fetch_search_results(keyword, limit=10):
    params = {
        "action": "query",
        "list": "search",
        "srsearch": keyword,
        "format": "json",
        "srlimit": limit
    }

    headers = {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(WIKI_API_URL, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()
        return data.get("query", {}).get("search", [])
    except requests.RequestException as e:
        print("検索リクエストでエラーが発生しました:", e)
        return []

# 指定されたタイトルのページ内容を取得する関数
def fetch_page_content(title):
    params = {
        "action": "query",
        "list": "search",
        "srsearch": title,
        "format": "json",
        "srlimit": 10
    }
    headers = {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(WIKI_API_URL, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()
        page = next(iter(data.get("query", {}).get("pages", {}).values()), {})
        return page.get("extract", "ページ内容が見つかりません。")
    except requests.RequestException as e:
        print("ページ内容取得でエラーが発生しました:", e)
        return "ページ内容が取得できませんでした。"
    

# キーワードに該当するページ一覧を取得
search_results = fetch_search_results(keyword)
if search_results:
    print("【キーワードに該当するページ一覧】")
    for result in search_results[:3]:
        print(f"- {result['title']}")

    # 最初のページタイトルを取得
    first_title = search_results[0]["title"]
    print("\n【最初のページタイトル】")
    print(first_title)

    # 最初のページの内容を取得
    page_content = fetch_page_content(first_title)
    print("\n【ページ内容】")
    print(page_content[:500])
else:
    print("検索結果が見つかりません。")


# キーワードに該当するページ一覧を取得
search_results = fetch_search_results(keyword)
if search_results:
    print("【キーワードに該当するページ一覧】")
    for result in search_results[:3]:
        print(f"- {result['title']}")

    # 最初のページタイトルを取得
    first_title = search_results[0]["title"]
    print("\n【最初のページタイトル】")
    print(first_title)

    # 最初のページの内容を取得
    page_content = fetch_page_content(first_title)
    print("\n【ページ内容】")
    print(page_content[:500])
else:
    print("検索結果が見つかりません。")

# 要約を行うプロンプトを作成
prompt = f"""
以下の文章を要約してください。

# 条件：fetch_search_results
- 小学生にもわかるように
- 300文字程度

# 文章：
{page_content[:1000]}
"""

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt},
    ],
    max_tokens=500,
    temperature=0.3
)


【キーワードに該当するページ一覧】
- NBA
- NBAゲータレード・リーグ
- NBAオールルーキーチーム

【最初のページタイトル】
NBA

【ページ内容】
ページ内容が見つかりません。
【キーワードに該当するページ一覧】
- NBA
- NBAゲータレード・リーグ
- NBAオールルーキーチーム

【最初のページタイトル】
NBA

【ページ内容】
ページ内容が見つかりません。
